# Exercise 5

Using the files *transition_probabilities.csv* and *ovservation_likelihoods.csv*, compute the likelihood of POS tags for the following word sequence:

    I want to race

## Task 1

Load the given files and:

a) Write a function to calculate all possible tag sequences for the given word sentence.

In [63]:
import pandas as pd
import itertools
import operator

df_obs = pd.read_csv("observation_likelihoods.csv", ",", index_col=0)
df_obs = df_obs.rename(columns=lambda x : x.strip())

df_trans = pd.read_csv("transisition_probabilities.csv", ",", index_col=0)
df_trans = df_trans.rename(columns=lambda x : x.strip())
sentence = "I want to race"
display(df_trans)
display(df_obs)
def possible_tag_seq(probability_df_obs, sentence):
    return list(itertools.product(list(probability_df_obs.index), repeat=len(sentence.split())))

sequences = possible_tag_seq(df_obs, sentence)
print(sequences)

,VB,TO,NN,PPSS
<s>,0.0190,0.00430,0.04100,0.06700
VB,0.0038,0.03500,0.04700,0.00700
TO,0.8300,0.00000,0.00047,0.00000
NN,0.0040,0.01600,0.08700,0.00450
PPSS,0.2300,0.00079,0.00120,0.00014


,I,want,to,race
VB,0.00,0.009300,0.00,0.00012
TO,0.00,0.000000,0.99,0.00000
NN,0.00,0.000054,0.00,0.00057
PPSS,0.37,0.000000,0.00,0.00000


[('VB', 'VB', 'VB', 'VB'), ('VB', 'VB', 'VB', 'TO'), ('VB', 'VB', 'VB', 'NN'), ('VB', 'VB', 'VB', 'PPSS'), ('VB', 'VB', 'TO', 'VB'), ('VB', 'VB', 'TO', 'TO'), ('VB', 'VB', 'TO', 'NN'), ('VB', 'VB', 'TO', 'PPSS'), ('VB', 'VB', 'NN', 'VB'), ('VB', 'VB', 'NN', 'TO'), ('VB', 'VB', 'NN', 'NN'), ('VB', 'VB', 'NN', 'PPSS'), ('VB', 'VB', 'PPSS', 'VB'), ('VB', 'VB', 'PPSS', 'TO'), ('VB', 'VB', 'PPSS', 'NN'), ('VB', 'VB', 'PPSS', 'PPSS'), ('VB', 'TO', 'VB', 'VB'), ('VB', 'TO', 'VB', 'TO'), ('VB', 'TO', 'VB', 'NN'), ('VB', 'TO', 'VB', 'PPSS'), ('VB', 'TO', 'TO', 'VB'), ('VB', 'TO', 'TO', 'TO'), ('VB', 'TO', 'TO', 'NN'), ('VB', 'TO', 'TO', 'PPSS'), ('VB', 'TO', 'NN', 'VB'), ('VB', 'TO', 'NN', 'TO'), ('VB', 'TO', 'NN', 'NN'), ('VB', 'TO', 'NN', 'PPSS'), ('VB', 'TO', 'PPSS', 'VB'), ('VB', 'TO', 'PPSS', 'TO'), ('VB', 'TO', 'PPSS', 'NN'), ('VB', 'TO', 'PPSS', 'PPSS'), ('VB', 'NN', 'VB', 'VB'), ('VB', 'NN', 'VB', 'TO'), ('VB', 'NN', 'VB', 'NN'), ('VB', 'NN', 'VB', 'PPSS'), ('VB', 'NN', 'TO', 'VB'), ('V

b) Write a function to select the most probable tag sequence.

In [2]:
def most_prob(observation, transition, sequences, sentence):
    dic = {}
    tokens = sentence.split()  
    for sequence in sequences:
        val = 1
        for i in range(0, len(sequence)):
            if i == 0:
                val = val * observation.at[sequence[i], tokens[i]] * transition.at['<s>', sequence[i]]
            else:
                val = val * observation.at[sequence[i], tokens[i]] * transition.at[sequence[i-1], sequence[i]]
        dic[sequence] = val
    return max(dic.items(), key=operator.itemgetter(1))

print(most_prob(df_obs, df_trans, sequences, sentence))

(('PPSS', 'VB', 'TO', 'VB'), 1.8299949392339997e-10)


c) Discuss your solution.

The function possible_tag_seq recieves a sentence and a data farme with all possible tags and their probabilities. As the probabilites are not relevant for the task, the function calculates the number of tokens in the sentence by using the split funciton and counting (this only works if the resulting tokens match those of the file). The function then takes all possible POS-Tag  retrieved from the DataFrame and using Itertools calcualtes all possible permutaitons of these POS-Tags with a length equal to the number of tokens by calculating the Cartesian Product.

The function most_prob recieves two DataFrames, one cotaining the observation probabilites and the other containing the transformation possilities, as well as a list of all possible POS-Tag sequences and the given sentence. Using this information, the function iterates over all sequences in the passed list and calculates the probability of each on and saves it in a dicitonary. The probablity is calculated by multiplying the observation probabilites of the word with the transformation porbabilities. The function then returns the element in the dicitonary with the highes probability.

## Task 2

Load the given files and follow the algorithm shown in slide 13 of our Lab Session 5 to:

a) Compute the initialization step of Viterbi.

In [67]:
def initialize_viterbi(df_obs, df_trans, sent):
    tokens = sent.split()
    df = pd.DataFrame(index=df_trans.columns, columns=df_obs.columns)
    df_backpointer = pd.DataFrame(index=df_trans.columns, columns=df_obs.columns)
    for s in df_trans.columns:
        df.at[s, tokens[0]] = df_obs.at[s, tokens[0]] * df_trans.at['<s>', s]
        df_backpointer.at[s, tokens[0]] = 0
    df.set_index(df_obs.columns)
    return df, df_backpointer
viterbi_df, backpointer_df = initialize_viterbi(df_obs, df_trans, sentence)
display(viterbi_df, backpointer_df)

,I,want,to,race
VB,0,NaN,NaN,NaN
TO,0,NaN,NaN,NaN
NN,0,NaN,NaN,NaN
PPSS,0.02479,NaN,NaN,NaN


,I,want,to,race
VB,0,NaN,NaN,NaN
TO,0,NaN,NaN,NaN
NN,0,NaN,NaN,NaN
PPSS,0,NaN,NaN,NaN


b) Compute the recursion step of Viterbi.

In [78]:
def viterbi_rec(df, backpointer_df, df_obs, df_trans, sentence):
    tokens = sentence.split()
    for t in range(1, len(tokens)):
        for s in df_trans.columns:
            df.at[s, tokens[t]] = max(df[tokens[t-1]]) *  df_obs.at[s, tokens[t]] * df_trans.at[df[df[tokens[t-1]]==(df[tokens[t-1]].max())].index[0], s]
            backpointer_df.at[s, tokens[t]] = list(df[df[tokens[t-1]]==df[tokens[t-1]].max()].index) + [s]
    return df, backpointer_df
viterbi_df, backpointer_df = viterbi_rec(viterbi_df, backpointer_df, df_obs, df_trans, sentence)
display(viterbi_df, backpointer_df)

,I,want,to,race
VB,0,5.30258e-05,0,1.82999e-10
TO,0,0,1.83734e-06,0
NN,0,1.60639e-09,0,4.92225e-13
PPSS,0.02479,0,0,0


,I,want,to,race
VB,0,"[PPSS, VB]","[VB, VB]","[TO, VB]"
TO,0,"[PPSS, TO]","[VB, TO]","[TO, TO]"
NN,0,"[PPSS, NN]","[VB, NN]","[TO, NN]"
PPSS,0,"[PPSS, PPSS]","[VB, PPSS]","[TO, PPSS]"


c) Compute termination step of Viterbi.

In [117]:
def termination_viterbi(df, backpointer_df, sentence):
    display(backpointer_df)
    tokens = sentence.split()
    index = df[df[tokens[len(tokens)-1]]==df[tokens[len(tokens)-1]].max()].index[0]
    best_path_pointer = backpointer_df.at[index, tokens[len(tokens) - 1]]
    best_path = best_path_pointer
    for i in range(1, len(tokens) - 1):
        col = backpointer_df[tokens[len(tokens) - 1 - i]]
        for j in col:
            if j[1] == best_path[0]:
                best_path = [j[0]] + best_path
                break
    return df[tokens[len(tokens) - 1]].max(), best_path
print(termination_viterbi(viterbi_df, backpointer_df, sentence))

,I,want,to,race
VB,0,"[PPSS, VB]","[VB, VB]","[TO, VB]"
TO,0,"[PPSS, TO]","[VB, TO]","[TO, TO]"
NN,0,"[PPSS, NN]","[VB, NN]","[TO, NN]"
PPSS,0,"[PPSS, PPSS]","[VB, PPSS]","[TO, PPSS]"


(1.8299949392339997e-10, ['PPSS', 'VB', 'TO', 'VB'])


It would make a lot more sense to write the whole algorithm in one method, but beacuse of the way the task was given I split it into three different methods to be able to clearly split the three parts of the task. 